In [13]:
import pytse_client as tse
import numpy as np
import torch
import torch.nn as nn
# tickers = tse.download(symbols="all", write_to_csv=True)

Device = ("cuda"
          if torch.cuda.is_available()
          else "cpu"
)
print(f"Using {Device} device")


Using cuda device


In [14]:
sym="آباد"
ticker=tse.Ticker(sym)
length=ticker.history.shape[0]
train_length = int(np.floor(length*0.9))
test_length = length - train_length
days_history=100
days_predict=30

# Now we extract some portion of the first sequences of data for train
X_train=np.zeros([train_length - days_history - days_predict - 1 , days_history])
Y_train=np.zeros([train_length - days_history - days_predict - 1 , days_predict])
for i in range(train_length - days_history - days_predict - 1):
    m_pre = ticker.history.loc[i : i + days_history - 1].to_numpy()
    m_next = ticker.history.loc[i+1 : i + days_history].to_numpy()
    X_train[i,:] = (m_next[:,9]-m_pre[:,9])/m_pre[:,9] * 100
    m_pre = ticker.history.loc[i + days_history : i + days_history - 1 + days_predict].to_numpy()
    m_next = ticker.history.loc[i + days_history + 1 : i + days_history + days_predict].to_numpy()
    Y_train[i,:] = (m_next[:,9]-m_pre[:,9])/m_pre[:,9] * 100

# Now we extract some portion of the last sequences of data for test
X_test=np.zeros([test_length - days_predict - 1 , days_history])
Y_test=np.zeros([test_length - days_predict - 1 , days_predict])
for i in range(test_length - days_predict - 1):
    j = i + train_length - days_history
    m_pre = ticker.history.loc[j : j + days_history - 1].to_numpy()
    m_next = ticker.history.loc[j+1 : j + days_history].to_numpy()
    X_test[i,:] = (m_next[:,9]-m_pre[:,9])/m_pre[:,9] * 100
    m_pre = ticker.history.loc[j + days_history : j + days_history - 1 + days_predict].to_numpy()
    m_next = ticker.history.loc[j + days_history + 1 : j + days_history + days_predict].to_numpy()
    Y_test[i,:] = (m_next[:,9]-m_pre[:,9])/m_pre[:,9] * 100

train_arr = np.arange(train_length - days_history - days_predict - 1)
train_idx = np.random.permutation(train_arr)
X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

test_arr = np.arange(test_length - days_predict - 1)
test_idx = np.random.permutation(test_arr)
X_test = X_test[test_idx]
Y_test = Y_test[test_idx]
X_train = torch.from_numpy(X_train).to(device=Device,dtype=torch.float32)
Y_train = torch.from_numpy(Y_train).to(device=Device,dtype=torch.float32)

X_test = torch.from_numpy(X_test).to(device=Device,dtype=torch.float32)
Y_test = torch.from_numpy(Y_test).to(device=Device,dtype=torch.float32)

In [15]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        n=64
        super().__init__()
        self.Lin1 = nn.Linear(input_size,2*n)
        self.Lin2 = nn.Linear(2*n,4*n)
        self.Lin3 = nn.Linear(4*n,2*n)
        self.Lin4 = nn.Linear(2*n,n)
        self.Lin5 = nn.Linear(n,output_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.activation2 = nn.Tanh()
        self.drop = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.Lin1(x)
            
        x = self.activation2(x)
        x = self.drop(x) ###
        x = self.Lin2(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        x = self.Lin3(x)
        x = self.activation2(x)
        x = self.drop(x) ###
            

        x = self.Lin4(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        x = self.Lin5(x)
        x = self.drop(x) ###
        return x
gama=0.99
W_gama=torch.diag(torch.tensor([gama**(1*i) for i in range(days_predict)],device=Device))
model=Model(days_history,days_predict).to(device=Device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
LossFn = nn.MSELoss()


In [16]:
batch_size = 64
train_Losses=[]
test_Losses=[]
N_train = int(np.floor(X_train.size(0)/batch_size))
N_test = int(np.floor(X_test.size(0)/batch_size))

for epoch in range(300):
    model.train()
    train_loss=torch.tensor(0,dtype=torch.float32)
    for i in range(N_train):
        x = X_train[i*batch_size:(i+1)*batch_size,:]
        y = Y_train[i*batch_size:(i+1)*batch_size,:]
        optimizer.zero_grad()
        out = model(x)
        loss_train = LossFn(y,out)
        loss_train.backward()
        optimizer.step()
        train_loss += loss_train.detach().item()
    train_Losses.append( train_loss/N_train )
   
    model.eval()
    test_loss=torch.tensor(0,dtype=torch.float32)
    test_acc=torch.tensor(0,dtype=torch.float32)
    with torch.no_grad():
        for i in range(N_test):
            x = X_test[i*batch_size:(i+1)*batch_size,:]
            y = Y_test[i*batch_size:(i+1)*batch_size,:].detach()
            out = model(x)
            loss_test = LossFn(y,out)
            test_loss += loss_test.detach().item()
            temp=y-out
            test_acc += torch.sum(1 - torch.abs(y@W_gama-out@W_gama)/y@W_gama).detach().item()
    test_Losses.append( test_loss/N_test )
    accuracy = test_acc / (N_test*batch_size*days_predict) *100
    print(f'{epoch}: train loss is {train_Losses[epoch]} & test loss is {test_Losses[epoch]} & accuracy is %{accuracy}')

0: train loss is 9.869823455810547 & test loss is 39.126277923583984 & accuracy is %nan
1: train loss is 9.863832473754883 & test loss is 39.121742248535156 & accuracy is %nan
2: train loss is 9.874431610107422 & test loss is 39.1171989440918 & accuracy is %nan
3: train loss is 9.870155334472656 & test loss is 39.11210632324219 & accuracy is %nan
4: train loss is 9.870301246643066 & test loss is 39.1139030456543 & accuracy is %nan
5: train loss is 9.870537757873535 & test loss is 39.124778747558594 & accuracy is %nan
6: train loss is 9.870307922363281 & test loss is 39.123680114746094 & accuracy is %nan
7: train loss is 9.878641128540039 & test loss is 39.118988037109375 & accuracy is %nan
8: train loss is 9.88473892211914 & test loss is 39.14191818237305 & accuracy is %nan
9: train loss is 9.89375114440918 & test loss is 39.12363052368164 & accuracy is %nan
10: train loss is 9.875864028930664 & test loss is 39.12308120727539 & accuracy is %nan
11: train loss is 9.862142562866211 & tes

KeyboardInterrupt: 